In [1]:
# Imports

# pandas
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [3]:
from preprocess import preprocessData
[Dtrainmat_Diagnosis, Y_FutureDiagnosis, RID, Dtrainmat, Dtrain] = preprocessData()



Load data and select features


/Users/danielwu/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,7

Train SVM for Diagnosis and SVR for ADAS and Ventricles


Using TensorFlow backend.


[MICE] Completing matrix with shape (12012, 22)
[MICE] Starting imputation round 1/110, elapsed time 0.004
[MICE] Starting imputation round 2/110, elapsed time 0.130
[MICE] Starting imputation round 3/110, elapsed time 0.178
[MICE] Starting imputation round 4/110, elapsed time 0.230
[MICE] Starting imputation round 5/110, elapsed time 0.278
[MICE] Starting imputation round 6/110, elapsed time 0.323
[MICE] Starting imputation round 7/110, elapsed time 0.367
[MICE] Starting imputation round 8/110, elapsed time 0.412
[MICE] Starting imputation round 9/110, elapsed time 0.459
[MICE] Starting imputation round 10/110, elapsed time 0.504
[MICE] Starting imputation round 11/110, elapsed time 0.549
[MICE] Starting imputation round 12/110, elapsed time 0.594
[MICE] Starting imputation round 13/110, elapsed time 0.640
[MICE] Starting imputation round 14/110, elapsed time 0.689
[MICE] Starting imputation round 15/110, elapsed time 0.735
[MICE] Starting imputation round 16/110, elapsed time 0.780
[

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Flatten,BatchNormalization, PReLU, Activation, Lambda, ELU, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.constraints import maxnorm
import keras.utils

Y = Y_FutureDiagnosis.as_matrix()


def nn_model():
    model = Sequential()
    model.add(Dense(500,input_shape=(22,), kernel_initializer = 'he_normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(300, kernel_initializer="he_normal", kernel_constraint=maxnorm(3)))
    model.add(PReLU())
    model.add(Dropout(0.2))
    #     model.add(Embedding(input_dim=18, output_dim=128))
    #     model.add(LSTM(units=128))
    model.add(Dense(3, activation='softmax', kernel_initializer='he_normal'))

    model.compile(optimizer='Adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return(model)


In [13]:
def createBaggingModels(n):
    """Helper function to create a dictionary of submodels for use in the bagging training"""
    bagModels = {}
    for i in range(n):
        name = "mlp" + str(i)
        print(name)
        bagModels[name] = nn_model()
        
    return bagModels

In [14]:
def run_network(xtrain,ytrain, model=None, epochs=40, batch=64):
    start_time = time.time()

    one_hot_labels = keras.utils.to_categorical(ytrain - 1, num_classes=3)    
    print ('Training model...')
    model.fit(xtrain, one_hot_labels, epochs=40, batch_size=64, verbose=2)

    print ("Training duration : {0}".format(time.time() - start_time))
    #score = model.evaluate(X_test, y_test, batch_size=16)

    #print ("Network's test score [loss, accuracy]: {0}".format(score))
    return model

In [15]:
# Train all the models
subModelCount = 10
models = createBaggingModels(subModelCount)
print(Y)
import time
X_train = Dtrainmat_Diagnosis
toTrain = models.keys()
bootstrapTrainSize = len(X_train) * 1
for k in toTrain:
    print("Training model: " + k)
    trainNdx = np.random.choice(range(len(X_train)), int(bootstrapTrainSize))
    m = run_network(X_train[trainNdx],Y[trainNdx], models[k])
    models[k].save_weights("model-" + k + ".hdf5", overwrite=True)

mlp0
mlp1
mlp2
mlp3
mlp4
mlp5
mlp6
mlp7
mlp8
mlp9
[ 1.  1.  1. ...,  1.  1.  1.]
Training model: mlp0
Training model...
Epoch 1/40
0s - loss: 0.8162 - acc: 0.6035
Epoch 2/40
0s - loss: 0.7029 - acc: 0.6564
Epoch 3/40
0s - loss: 0.6635 - acc: 0.6801
Epoch 4/40
0s - loss: 0.6318 - acc: 0.7020
Epoch 5/40
0s - loss: 0.6076 - acc: 0.7171
Epoch 6/40
0s - loss: 0.5827 - acc: 0.7263
Epoch 7/40
0s - loss: 0.5690 - acc: 0.7408
Epoch 8/40
0s - loss: 0.5395 - acc: 0.7527
Epoch 9/40
0s - loss: 0.5204 - acc: 0.7697
Epoch 10/40
0s - loss: 0.5032 - acc: 0.7779
Epoch 11/40
0s - loss: 0.4825 - acc: 0.7941
Epoch 12/40
0s - loss: 0.4561 - acc: 0.8023
Epoch 13/40
0s - loss: 0.4446 - acc: 0.8141
Epoch 14/40
0s - loss: 0.4293 - acc: 0.8159
Epoch 15/40
0s - loss: 0.4052 - acc: 0.8296
Epoch 16/40
0s - loss: 0.3868 - acc: 0.8423
Epoch 17/40
0s - loss: 0.3802 - acc: 0.8390
Epoch 18/40
0s - loss: 0.3612 - acc: 0.8531
Epoch 19/40
0s - loss: 0.3490 - acc: 0.8606
Epoch 20/40
0s - loss: 0.3339 - acc: 0.8623
Epoch 21/

0s - loss: 0.3477 - acc: 0.8582
Epoch 20/40
0s - loss: 0.3317 - acc: 0.8650
Epoch 21/40
0s - loss: 0.3156 - acc: 0.8749
Epoch 22/40
0s - loss: 0.3014 - acc: 0.8772
Epoch 23/40
0s - loss: 0.2890 - acc: 0.8841
Epoch 24/40
0s - loss: 0.2852 - acc: 0.8866
Epoch 25/40
0s - loss: 0.2753 - acc: 0.8932
Epoch 26/40
0s - loss: 0.2692 - acc: 0.8957
Epoch 27/40
0s - loss: 0.2542 - acc: 0.9019
Epoch 28/40
0s - loss: 0.2479 - acc: 0.9047
Epoch 29/40
0s - loss: 0.2380 - acc: 0.9119
Epoch 30/40
0s - loss: 0.2272 - acc: 0.9132
Epoch 31/40
0s - loss: 0.2307 - acc: 0.9098
Epoch 32/40
0s - loss: 0.2141 - acc: 0.9190
Epoch 33/40
0s - loss: 0.2110 - acc: 0.9218
Epoch 34/40
0s - loss: 0.2007 - acc: 0.9256
Epoch 35/40
0s - loss: 0.1925 - acc: 0.9322
Epoch 36/40
0s - loss: 0.1947 - acc: 0.9309
Epoch 37/40
0s - loss: 0.1737 - acc: 0.9347
Epoch 38/40
0s - loss: 0.1812 - acc: 0.9319
Epoch 39/40
0s - loss: 0.1694 - acc: 0.9360
Epoch 40/40
0s - loss: 0.1596 - acc: 0.9461
Training duration : 26.011595010757446
Train

0s - loss: 0.1687 - acc: 0.9411
Epoch 40/40
0s - loss: 0.1611 - acc: 0.9423
Training duration : 26.283725023269653
Training model: mlp9
Training model...
Epoch 1/40
1s - loss: 0.8091 - acc: 0.6082
Epoch 2/40
0s - loss: 0.6924 - acc: 0.6658
Epoch 3/40
0s - loss: 0.6541 - acc: 0.6841
Epoch 4/40
0s - loss: 0.6245 - acc: 0.7102
Epoch 5/40
0s - loss: 0.6001 - acc: 0.7267
Epoch 6/40
0s - loss: 0.5844 - acc: 0.7387
Epoch 7/40
0s - loss: 0.5504 - acc: 0.7517
Epoch 8/40
0s - loss: 0.5309 - acc: 0.7653
Epoch 9/40
0s - loss: 0.5114 - acc: 0.7750
Epoch 10/40
0s - loss: 0.4901 - acc: 0.7954
Epoch 11/40
0s - loss: 0.4733 - acc: 0.7976
Epoch 12/40
0s - loss: 0.4540 - acc: 0.8121
Epoch 13/40
0s - loss: 0.4366 - acc: 0.8184
Epoch 14/40
0s - loss: 0.4247 - acc: 0.8185
Epoch 15/40
0s - loss: 0.4010 - acc: 0.8372
Epoch 16/40
0s - loss: 0.3925 - acc: 0.8346
Epoch 17/40
0s - loss: 0.3799 - acc: 0.8407
Epoch 18/40
0s - loss: 0.3534 - acc: 0.8561
Epoch 19/40
0s - loss: 0.3404 - acc: 0.8622
Epoch 20/40
0s - lo

In [16]:
import os

    
str_exp = '/Users/danielwu/Dropbox/Documents/CSCI5525/Project/'
S = pd.read_csv(os.path.join(str_exp, 'IntermediateData',
                             'ToPredict.csv'), header=None)
S = S.values

Dtestmat = np.zeros((len(S), Dtrainmat.shape[1]))
# y_Dtestmat = np.zeros((len(S),1))
for i in range(len(S)):
    idx_S = RID.values == S[i]
    Dtestmat[i, :] = Dtrainmat[np.where(idx_S)[0][-1], :]
#     y_Dtestmat[i] = Dtrainmat[np.where(idx_S)[0][-1], :]

# mt = []
# st = []
# for i in range(Dtrainmat.shape[1]):
#     mt.append(np.nanmean(Dtestmat[:, i]))
#     st.append(np.nanstd(Dtestmat[:, i]))
#     Dtestmat[np.isnan(Dtestmat[:, i]), i] = mt[i]
#     Dtestmat[:, i] = (Dtestmat[:, i] - mt[i]) / st[i]

    
print("Running predictions on test set samples...")
results = np.zeros((len(S),3))
#print(np.shape(results))
for k in models.keys():
    print("Model: " + k)
    p = models[k].predict(Dtestmat)
    results += p
    print(np.shape(p))

results = results / subModelCount
# print(results)

# Write ouput format to files
o = np.column_stack((S, S, S, results))
count = 0
years = [str(a) for a in range(2010, 2018)]
months = [str(a).zfill(2) for a in range(1, 13)]
ym = [y + '-' + mo for y in years for mo in months]
ym = ym[4:-8]
nr_pred = len(ym)
o1 = np.zeros((o.shape[0] * nr_pred, o.shape[1]))
ym1 = [a for b in range(0, len(S)) for a in ym]
for i in range(len(o)):
    o1[count:count + nr_pred] = o[i]
    o1[count:count + nr_pred, 1] = range(1, nr_pred + 1)
    count = count + nr_pred

output = pd.DataFrame(o1, columns=['RID', 'Forecast Month', 'Forecast Date','CN relative probability', 'MCI relative probability', 'AD relative probability'])
output['Forecast Month'] = output['Forecast Month'].astype(int)
output['Forecast Date'] = ym1

str_out_final = os.path.join(str_exp, 'IntermediateData', 'TADPOLE_Submission_Leaderboard_NeuralNet.csv')
output.to_csv(str_out_final, header=True, index=False)

print('Evaluate predictions')
R = pd.read_csv('./TADPOLE_LB4.csv')
import evalOneSubmission as eos
mAUC, bca = eos.evalOneSub(R, output)

print('Diagnosis:')
print('mAUC = ' + "%0.3f" % mAUC)
print('BAC = ' + "%0.3f" % bca)


Running predictions on test set samples...
Model: mlp0
(110, 3)
Model: mlp1
(110, 3)
Model: mlp2
(110, 3)
Model: mlp3
(110, 3)
Model: mlp4
(110, 3)
Model: mlp5
(110, 3)
Model: mlp6
(110, 3)
Model: mlp7
(110, 3)
Model: mlp8
(110, 3)
Model: mlp9
(110, 3)
Evaluate predictions
Diagnosis:
mAUC = 0.645
BAC = 0.566


In [ ]:
def BaggingAverage(results, verbose = False):
    """Average the probabilities for a given test sample across models to finalize the prediction""" 
    correct = 0.0
    missedTests = []
    predictionMisses = []
    for i in range(samples):
        gs = np.argmax(y_test[i])
        ra = np.zeros(3)
        for r in reuslts:
            ra = += r[i]
        f = ra/ len(results)
        prd = np.argmax(f)
        frnd = np.round(f,5)
    if(prd == gs):
        correct += 1
    else:
        if(verbose):
            for r in results:
                print(np.round(r[i], 5))
            print("Agg: " + str(frnd))
            print("Predicted: " + str(prd))
            print("GoldStd:   " + str(gs))
        missedTests.append(X_test[i])
        predictionMisses.append(prd)
    
    return correct

def BaggingVote(results, verbose = False):
    """Vote based on the probabilities for a given test sample across models to finalize the prediction"""
    correct = 0.0
    
    missedTests = []
    predictionMisses = []
    for i in range(samples):
        #print(y_test[i])
        gs = np.argmax(y_test[i])
        votes = np.zeros(3)
        for r in results:
            # Use argmax to get the primary vote from this model and increment the cooresponding slot 
            # in the aggregated result
            vote = np.argmax(r[i])
            #print(vote)
            votes[vote] += 1
        # Use argmax again to extract the slot that has the most votes.
        prd = np.argmax(votes)

        if(prd == gs):
            correct += 1
        else:
            if(verbose):
                for r in results:
                    print(np.round(r[i], 5))
                print("Votes: " + str(votes))
                print("Predicted: " + str(prd))
                print("GoldStd:   " + str(gs))
            missedTests.append(X_test[i])
            predictionMisses.append(prd)
    
    return correct

In [ ]:
#Write ouput format to files
o=np.column_stack((S,S,S,p))
count=0
years=[str(a) for a in range(2010,2018)]
months=[str(a).zfill(2) for a in range(1,13)]
ym=[y + '-' + mo for y in years for mo in months ]
ym=ym[4:-8]
nr_pred=len(ym)
o1 = np.zeros((o.shape[0]*nr_pred,o.shape[1]))
ym1 = [a for b in range(0, len(S)) for a in ym ]
for i in range(len(o)):
    o1[count:count+nr_pred]=o[i]
    o1[count:count+nr_pred,1]=range(1,nr_pred+1)
    count=count+nr_pred
    

output=pd.DataFrame(o1, columns=['RID','Forecast Month','Forecast Date','CN relative probability','MCI relative probability','AD relative probability'])
output['Forecast Month'] = output['Forecast Month'].astype(int)
output['Forecast Date'] = ym1

str_out_final = os.path.join(str_exp, 'IntermediateData','TADPOLE_Submission_Leaderboard_BenchmarkSVM.csv')
output.to_csv(str_out_final,header=True,index=False)


print('Evaluate predictions')
R=pd.read_csv('./TADPOLE_LB4.csv')
import evalOneSubmission as eos
mAUC, bca = eos.evalOneSub(R,output)

print('Diagnosis:')
print('mAUC = ' + "%0.3f" % mAUC,)
print ('BAC = ' + "%0.3f" % bca)

In [ ]:
# What is our percentage success from averaging?
print ("Bagging w/ Averaging")
print ("--------------------")
correct = BaggingAverage(results)
print ("Misses: " + str(samples - correct))
print correct/samples

# Separator...
print
print

# What is our percentage success from voting?
print ("Bagging w/ Voting")
print ("-----------------")
correctVote = BaggingVote(results)
print ("Misses: " + str(samples - correctVote))